# Smartphone capturing by IP Webcam

This code access to smartphone video stream.  
You need to install [IP Webcam] in your smartphone, and "Start server" using it.

[IP Webcam]:https://play.google.com/store/apps/details?id=com.pas.webcam

In [1]:
import shutil, os
from datetime import datetime

import numpy as np
import cv2
import urllib.request

In [2]:
folder_root = "C:/FurImages/Capture"

## load url from text file ...
url = "http://192.168.0.1:8080"
with open('data/ipaddress.txt') as f: 
    url = f.read()

## 
url_video    = url+"/video"
url_snapshot = url+"/photoaf.jpg"

## Main loop

Now it shows video in real-time to check capturing.  
You can control this tool with the following keyboard.

- Escape: quit loop
- Space bar: capture the video view (**low-resolution**)
- 'c' key: capture the image (**high-resolution**)

In [3]:
############################################################
## create folder for captured images
############################################################
datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_path  = "{0}/capture_{1}".format(folder_root, datetime_str)
os.makedirs(folder_path)

############################################################
## main loop
############################################################
capture = cv2.VideoCapture(url_video) ## start video preview
count = 0 # initialize counter for image capturing

while True:
    
    ## read & show image from stream
    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)
    key = cv2.waitKey(1)
    
    ## ESCAPE: abort
    if key==27: break
    
    ########################################
    ## SPACEBAR: screenshot (low-res)
    ########################################
    if key==ord(' '):
        img=frame
        cv2.imshow('capture_low-res',img)
    
        ## write full-sized image in PNG without compression
        img_path = "{0}/capture_{1:04d}.png".format(folder_path, count)
        cv2.imwrite(img_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        count+=1
    
    ########################################
    ## 'c' key: screenshot (hi-res)
    ########################################
    if key==ord('c'):
        with urllib.request.urlopen(url_snapshot) as url:
            
            ## get snapshot from IP Webcam
            imgNp=np.array(bytearray(url.read()),dtype=np.uint8)
            img=cv2.imdecode(imgNp,-1)
            
            ## show temporary, resized view
            H, W = img.shape[:2]
            W_small = 640
            H_small = int(H / (W/W_small))
            img_small = cv2.resize(img, (W_small,H_small) )
            cv2.imshow('capture_hi-res',img_small)
            
            ## write full-sized image in PNG without compression
            img_path = "{0}/capture_{1:04d}_HQ.png".format(folder_path, count)
            cv2.imwrite(img_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            count+=1

capture.release()
cv2.destroyAllWindows()